In [ ]:
from PIL import Image

def preprocess_image(image_path, size=(128, 128)):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        img = img.resize(size)
    return img

In [ ]:
import numpy as np
import imagehash
from imagehash import ImageHash

def block_hash(img, block_size=4):
    img = img.convert("L")
    width, height = img.size
    pixels = np.array(img)
    
    block_values = []
    
    for i in range(0, width, block_size):
        for j in range(0, height, block_size):
            block = pixels[i:i+block_size, j:j+block_size]
            block_values.append(np.mean(block))
            
    image_mean = np.mean(pixels)
    hash_bits = [1 if value > image_mean else 0 for value in block_values]
    
    hash_array = np.array([int(bit) for bit in map(str, hash_bits)])
    return ImageHash(hash_array.reshape(int(width/block_size), int(height/block_size)))

def median_hash(img):
    img = img.convert("L")
    pixels = np.array(img).flatten()
    
    median_value = np.median(pixels)
    hash_bits = [1 if pixel > median_value else 0 for pixel in pixels]
    
    hash_array = np.array([int(bit) for bit in map(str, hash_bits)])
    return ImageHash(hash_array.reshape(img.size))

In [ ]:
def compute_hashes(image):
    ahash = imagehash.average_hash(image)
    bhash = block_hash(image)
    dhash = imagehash.dhash(image)
    mhash = median_hash(image)
    whash = imagehash.whash(image)
    phash = imagehash.phash(image)
    
    return ahash, bhash, dhash, mhash, whash, phash

In [ ]:
def hamming_distance_to_similarity(distance, hash_length):
    similarity = (hash_length - distance) / hash_length
    return similarity

def compare_hashes(hashes1, hashes2):
    similarities = {}
    labels = ['average', 'block', 'dhash', 'median', 'whash', 'phash']
    for (h1, h2, label) in zip(hashes1, hashes2, labels):
        distance = h1 - h2
        similarity = hamming_distance_to_similarity(distance, len(h1.hash)**2)
        similarities[label] = similarity
    return similarities

def compare_images(img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)
    
    hashes1 = compute_hashes(img1)
    hashes2 = compute_hashes(img2)
    
    return compare_hashes(hashes1, hashes2)

In [ ]:
img1_path = "path_to_image1.jpg"
img2_path = "path_to_image2.jpg"

similarities = compare_images(img1_path, img2_path)
for method, similarity in similarities.items():
    print(f"Similarity using {method}: {similarity:.4f}")

In [10]:
from PIL import Image
import numpy as np
import imagehash
from imagehash import ImageHash

def preprocess_image(image_path, size=(128, 128)):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        img = img.resize(size)
    return img

def block_hash(img, block_size=4):
    img = img.convert("L")
    width, height = img.size
    pixels = np.array(img)
    
    block_values = []
    
    for i in range(0, width, block_size):
        for j in range(0, height, block_size):
            block = pixels[i:i+block_size, j:j+block_size]
            block_values.append(np.mean(block))
            
    image_mean = np.mean(pixels)
    hash_bits = [1 if value > image_mean else 0 for value in block_values]
    
    hash_array = np.array([int(bit) for bit in map(str, hash_bits)])
    return ImageHash(hash_array.reshape(int(width/block_size), int(height/block_size)))

def median_hash(img):
    img = img.convert("L")
    pixels = np.array(img).flatten()
    
    median_value = np.median(pixels)
    hash_bits = [1 if pixel > median_value else 0 for pixel in pixels]
    
    hash_array = np.array([int(bit) for bit in map(str, hash_bits)])
    return ImageHash(hash_array.reshape(img.size))

def compute_hashes(image):
    ahash = imagehash.average_hash(image)
    bhash = block_hash(image)
    dhash = imagehash.dhash(image)
    mhash = median_hash(image)
    whash = imagehash.whash(image)
    phash = imagehash.phash(image)
    
    return ahash, bhash, dhash, mhash, whash, phash

def hamming_distance_to_similarity(distance, hash_length):
    similarity = (hash_length - distance) / hash_length
    return similarity

def compare_hashes(hashes1, hashes2):
    similarities = {}
    labels = ['average', 'block', 'dhash', 'median', 'whash', 'phash']
    for (h1, h2, label) in zip(hashes1, hashes2, labels):
        distance = h1 - h2
        similarity = hamming_distance_to_similarity(distance, len(h1.hash)**2)
        similarities[label] = similarity
    return similarities

def compare_images(img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)
    
    hashes1 = compute_hashes(img1)
    hashes2 = compute_hashes(img2)
    
    return compare_hashes(hashes1, hashes2)

In [11]:
img1_path = "nft_images/test_images/alpacadabraz.png"
img2_path = "nft_images/test_images/cryptoskulls.png"

similarities = compare_images(img1_path, img2_path)
for method, similarity in similarities.items():
    print(f"Similarity using {method}: {similarity:.4f}")

Similarity using average: 0.7656
Similarity using block: 0.7539
Similarity using dhash: 0.5781
Similarity using median: 0.4966
Similarity using whash: 0.8438
Similarity using phash: 0.5312


In [12]:
import time

def compute_hashes_with_time(image):
    methods = [imagehash.average_hash, block_hash, imagehash.dhash, median_hash, imagehash.whash, imagehash.phash]
    labels = ['average', 'block', 'dhash', 'median', 'whash', 'phash']
    hash_results = []
    time_results = {}

    for method, label in zip(methods, labels):
        start_time = time.time()
        hash_value = method(image)
        end_time = time.time()
        elapsed_time = end_time - start_time
        time_results[label] = elapsed_time
        hash_results.append(hash_value)
    
    return hash_results, time_results

def compare_images_with_time(img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)
    
    hashes1, time_results1 = compute_hashes_with_time(img1)
    hashes2, time_results2 = compute_hashes_with_time(img2)

    similarities = compare_hashes(hashes1, hashes2)
    
    return similarities, time_results1, time_results2

similarities, time_img1, time_img2 = compare_images_with_time(img1_path, img2_path)
for method, similarity in similarities.items():
    print(f"Similarity using {method}: {similarity:.4f}")

print("\n시간 측정 결과:")
for method, time_taken in time_img1.items():
    print(f"Image 1 - {method}: {time_taken:.6f} seconds")

for method, time_taken in time_img2.items():
    print(f"Image 2 - {method}: {time_taken:.6f} seconds")

Similarity using average: 0.7656
Similarity using block: 0.7539
Similarity using dhash: 0.5781
Similarity using median: 0.4966
Similarity using whash: 0.8438
Similarity using phash: 0.5312

시간 측정 결과:
Image 1 - average: 0.000488 seconds
Image 1 - block: 0.010200 seconds
Image 1 - dhash: 0.000249 seconds
Image 1 - median: 0.027151 seconds
Image 1 - whash: 0.002369 seconds
Image 1 - phash: 0.000540 seconds
Image 2 - average: 0.000333 seconds
Image 2 - block: 0.008008 seconds
Image 2 - dhash: 0.000248 seconds
Image 2 - median: 0.027375 seconds
Image 2 - whash: 0.002391 seconds
Image 2 - phash: 0.000571 seconds
